# Multimodal Late Fusion Model
## Text + Tabular + Image embeddings -> SVM + XGBoost

## 0) Imports

In [11]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.svm import LinearSVC

from xgboost import XGBClassifier

import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Embedding, Bidirectional, LSTM,
    Dense, Dropout, BatchNormalization,
    GlobalAveragePooling1D, GlobalMaxPooling1D,
    Concatenate
)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

## 1) Configuration

In [2]:
df = pd.read_csv("FoodFactsCleaned.csv")
df["nutriscore_letter"] = df["nutriscore_letter"].astype(int)

In [3]:
TEXT_COLS = [ 
    "brand_cleaned",
    "allergens_cleaned",
    "ingredients_text_cleaned",
    "countries_cleaned",
    "additives_cleaned",
]

TABULAR_COLS = [
    'nova_group', 'fat_100g',
    'saturated_fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g',
    'proteins_100g', 'contains_palm_oil', 'vegetarian_status', 'vegan_status',
    'nutrient_level_fat', 'nutrient_level_saturated_fat',
    'nutrient_level_sugars', 'nutrient_level_salt', 'ecoscore_grade', 'ecoscore_score',
    'carbon_footprint_100g', 'additives_count', 'sugar_ratio',
    'energy_density', 'protein_ratio', 'macro_balance', 'healthy_score',
    'log_energy_kcal_100g', 'log_salt_100g'
]

TARGET_COL = "nutriscore_letter"           
IMAGE_COL = "image_160_path"

RANDOM_STATE = 42
TEST_SIZE = 0.2

# Text tokenization
MAX_WORDS = 30000
MAX_LEN = 200

# Image settings
IMG_SIZE = (160, 160)

# NN training
EPOCHS_TEXT = 10
EPOCHS_TAB  = 25
EPOCHS_IMG  = 10
BATCH_SIZE = 64

## 2) Basic checks + building concatenated text

In [4]:
needed = TEXT_COLS + TABULAR_COLS + [TARGET_COL, IMAGE_COL]
missing = [c for c in needed if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns in df: {missing}")

for c in TEXT_COLS:
    df[c] = df[c].fillna("").astype(str)

df["text_concat"] = df[TEXT_COLS].agg(" ".join, axis=1)

df = df[df[IMAGE_COL].notna()].copy()
df = df[df[IMAGE_COL].astype(str).str.len() > 0].copy()
df = df[df[IMAGE_COL].apply(lambda p: os.path.exists(str(p)))].copy()

print("Rows after filtering invalid image paths:", len(df))

Rows after filtering invalid image paths: 5138


## 3) Unified Data Splitting across Modalities

In [6]:
X_text = df["text_concat"].values
X_tab  = df[TABULAR_COLS].values.astype(np.float32)
X_img  = df[IMAGE_COL].astype(str).values
y_raw  = df[TARGET_COL].values

if y_raw.dtype == object or isinstance(y_raw[0], str):
    le = LabelEncoder()
    y = le.fit_transform(y_raw)
else:
    y = y_raw.astype(int)

num_classes = len(np.unique(y))
print("Classes:", num_classes)

X_text_tr, X_text_te, X_tab_tr, X_tab_te, X_img_tr, X_img_te, y_tr, y_te = train_test_split(
    X_text, X_tab, X_img, y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=y
)

print("Train:", len(y_tr), "Test:", len(y_te))

Classes: 5
Train: 4110 Test: 1028


## Text & Tabular Data Preprocessing

In [7]:
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_text_tr)

seq_tr = tokenizer.texts_to_sequences(X_text_tr)
seq_te = tokenizer.texts_to_sequences(X_text_te)

X_text_tr_pad = pad_sequences(seq_tr, maxlen=MAX_LEN, padding="post", truncating="post")
X_text_te_pad = pad_sequences(seq_te, maxlen=MAX_LEN, padding="post", truncating="post")

vocab_size = min(MAX_WORDS, len(tokenizer.word_index) + 1)
print("Vocab size:", vocab_size)

scaler = StandardScaler()
X_tab_tr_sc = scaler.fit_transform(X_tab_tr).astype(np.float32)
X_tab_te_sc = scaler.transform(X_tab_te).astype(np.float32)

tab_dim = X_tab_tr_sc.shape[1]
print("Tab dim:", tab_dim)

Vocab size: 14981
Tab dim: 25


## Image loading & preprocessing

In [8]:
def load_and_preprocess_image(path):
    img_bytes = tf.io.read_file(path)
    img = tf.io.decode_image(img_bytes, channels=3, expand_animations=False)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32)
    img = mobilenet_preprocess(img)
    return img

def make_img_ds(paths, labels, batch_size=64, training=True):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))

    def _map(p, y_):
        return load_and_preprocess_image(p), y_
    
    ds = ds.map(_map, num_parallel_calls=tf.data.AUTOTUNE)
    if training:
        ds = ds.shuffle(2048, seed=RANDOM_STATE, reshuffle_each_iteration=True)

    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

img_train_ds = make_img_ds(X_img_tr, y_tr, batch_size=32, training=True)
img_test_ds  = make_img_ds(X_img_te, y_te, batch_size=32, training=False)

## 6) Modality-specific models

### 6A) TEXT model: BiLSTM + pooling -> embedding

In [13]:
def build_text_model(vocab_size, max_len, num_classes, embed_dim=128, rnn_units=64):
    text_in = Input(shape=(max_len,), dtype=tf.int32, name="text_in")

    x = Embedding(vocab_size, embed_dim, name="txt_embed")(text_in)
    x = SimpleRNN(rnn_units,return_sequences=True, name="txt_rnn")(x)

    avg_pool = GlobalAveragePooling1D(name="txt_gap")(x)
    max_pool = GlobalMaxPooling1D(name="txt_gmp")(x)
    x = Concatenate(name="txt_pool_concat")([avg_pool, max_pool])

    x = Dense(256, activation="relu", name="txt_embed_dense")(x)
    x = Dropout(0.35)(x)

    out = Dense(num_classes, activation="softmax", name="txt_out")(x)

    model = Model(inputs=text_in, outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

text_model = build_text_model(vocab_size, MAX_LEN, num_classes)
text_model.summary()

text_model.fit(
    X_text_tr_pad, y_tr,
    validation_split=0.2,
    epochs=EPOCHS_TEXT,
    batch_size=BATCH_SIZE,
    callbacks=[
        EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
        ReduceLROnPlateau(monitor="val_loss", patience=1, factor=0.5, min_lr=1e-6)
    ],
    verbose=1
)

text_extractor = Model(
    inputs=text_model.input,
    outputs=text_model.get_layer("txt_embed_dense").output
)

Z_txt_tr = text_extractor.predict(X_text_tr_pad, batch_size=256, verbose=0)
Z_txt_te = text_extractor.predict(X_text_te_pad, batch_size=256, verbose=0)

print("Text embeddings:", Z_txt_tr.shape, Z_txt_te.shape)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text_in             │ (None, 200)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ txt_embed           │ (None, 200, 128)  │  1,917,568 │ text_in[0][0]     │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ txt_rnn (SimpleRNN) │ (None, 200, 64)   │     12,352 │ txt_embed[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ txt_gap             │ (None, 64)        │          0 │ txt_rnn[0][0]     │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ txt_gmp             │ (None, 64)        │          0 │ txt_rnn[0][0]     │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ txt_pool_concat     │ (None, 128)       │          0 │ txt_gap[0][0],    │
│ (Concatenate)       │                   │            │ txt_gmp[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ txt_embed_dense     │ (None, 256)       │     33,024 │ txt_pool_concat[… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 256)       │          0 │ txt_embed_dense[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ txt_out (Dense)     │ (None, 5)         │      1,285 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,964,229 (7.49 MB)

 Trainable params: 1,964,229 (7.49 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 6s 71ms/step - accuracy: 0.3157 - loss: 1.4884 - val_accuracy: 0.3017 - val_loss: 1.4740 - learning_rate: 0.0010
Epoch 2/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 69ms/step - accuracy: 0.3817 - loss: 1.3829 - val_accuracy: 0.3929 - val_loss: 1.4204 - learning_rate: 0.0010
Epoch 3/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - accuracy: 0.4617 - loss: 1.2665 - val_accuracy: 0.4477 - val_loss: 1.3176 - learning_rate: 0.0010
Epoch 4/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 5s 66ms/step - accuracy: 0.5803 - loss: 1.0873 - val_accuracy: 0.5316 - val_loss: 1.1178 - learning_rate: 0.0010
Epoch 5/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - accuracy: 0.7138 - loss: 0.7847 - val_accuracy: 0.5535 - val_loss: 1.0472 - learning_rate: 0.0010
Epoch 6/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step - accuracy: 0.8345 - loss: 0.5287 - val_accuracy: 0.5779 - val_loss: 1.0724 - learning_rate: 0.0010
Epoch 7/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 62ms/step - accuracy: 0.9179 - loss: 0.3212 - val_acc